<a href="https://colab.research.google.com/github/nithin24011/Nithin_INFO5731_Spring2020/blob/main/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [1]:
# Write your code here
!pip install TextBlob

In [2]:
import pandas as pd
data = pd.read_csv("/content/joker_data1.csv")
data.head(10)
main_df=data[["review_content"]]
main_df.head(10)

,review_content
0,I was a person that saw all the hype and claim...
1,"Every once in a while a movie comes, that trul..."
2,This is a movie that only those who have felt ...
3,"Truly a masterpiece, The Best Hollywood film o..."
4,Most of the time movies are anticipated like t...
5,Joaquin Phoenix gives a tour de force performa...
6,Let me start off by saying if Joaquin Phoneix ...
7,I get why some people hate this . It's because...
8,I have seen Joker yesterday at Venice an early...
9,It's sad that Joaquin missed Oscar for 'The gl...


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#remove stop words 
from nltk.corpus import stopwords
stopwords = list(stopwords.words('english'))

#find the occurance of each word from the list
main_df["Tokens"] = main_df["review_content"].apply(lambda x : nltk.word_tokenize(str(x)))
main_df.head(10)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,review_content,Tokens
0,I was a person that saw all the hype and claim...,"[I, was, a, person, that, saw, all, the, hype,..."
1,"Every once in a while a movie comes, that trul...","[Every, once, in, a, while, a, movie, comes, ,..."
2,This is a movie that only those who have felt ...,"[This, is, a, movie, that, only, those, who, h..."
3,"Truly a masterpiece, The Best Hollywood film o...","[Truly, a, masterpiece, ,, The, Best, Hollywoo..."
4,Most of the time movies are anticipated like t...,"[Most, of, the, time, movies, are, anticipated..."
5,Joaquin Phoenix gives a tour de force performa...,"[Joaquin, Phoenix, gives, a, tour, de, force, ..."
6,Let me start off by saying if Joaquin Phoneix ...,"[Let, me, start, off, by, saying, if, Joaquin,..."
7,I get why some people hate this . It's because...,"[I, get, why, some, people, hate, this, ., It,..."
8,I have seen Joker yesterday at Venice an early...,"[I, have, seen, Joker, yesterday, at, Venice, ..."
9,It's sad that Joaquin missed Oscar for 'The gl...,"[It, 's, sad, that, Joaquin, missed, Oscar, fo..."


In [4]:
import pandas as pd
import numpy as np
def compute_values(ip):
  res1 =pd.value_counts(ip)
  if(len(res1)==0 ):
    res1 = [0]
  return res1

#calculate the occurance of each word in the sentence
df1 = main_df["Tokens"].apply(lambda x : compute_values(x))

In [10]:
df1 = df1.fillna(0)
final_df = pd.DataFrame(df1.columns, columns=["Tokens"])
res = pd.DataFrame(df1.T.sum(axis=1),columns = ["value"]).values.tolist()
final_df["value_of_tf"] = pd.DataFrame(res)

In [11]:
final_df.head(10)


,Tokens,value_of_tf
0,and,494.0
1,.,726.0
2,the,719.0
3,a,484.0
4,of,377.0
5,is,347.0
6,was,173.0
7,to,381.0
8,that,214.0
9,",",674.0


In [12]:
from textblob  import TextBlob
#find the polarity 
final_df["Polarity"] = final_df['Tokens'].apply(lambda x: TextBlob(x).sentiment.polarity)

#filter the influencing variables 
iv = final_df["Polarity"] != 0
iv_filter = final_df[iv]
iv_filter.head(10)

,Tokens,value_of_tf,Polarity
19,best,27.0,1.000
38,pretentious,3.0,-0.300
43,bleak,2.0,-1.000
45,rare,1.0,0.300
47,brilliant,9.0,0.900
48,extremely,3.0,-0.125
55,proud,1.0,0.800
64,impeccable,1.0,0.750
67,incredibly,3.0,0.900
68,magnificently,1.0,1.000


In [14]:
#ranks based on the occurance 
result = iv_filter.sort_values(by = ["value_of_tf"],ascending = False).reset_index()
result["Rank"] = pd.DataFrame(range(1,242))
result.head(10)

,index,Tokens,value_of_tf,Polarity,Rank
0,436,good,54.0,0.700,1.0
1,282,more,52.0,0.500,2.0
2,87,very,40.0,0.200,3.0
3,328,many,33.0,0.500,4.0
4,250,much,32.0,0.200,5.0
5,531,really,29.0,0.200,6.0
6,19,best,27.0,1.000,7.0
7,207,other,26.0,-0.125,8.0
8,293,bad,24.0,-0.700,9.0
9,434,most,24.0,0.500,10.0


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [25]:
# Write your code here
import pandas as pd
df=pd.read_csv("/content/joker_data1.csv",usecols=["review_content","Sentiment"])
df=df.dropna().reset_index()
df

,index,review_content,Sentiment
0,0,I was a person that saw all the hype and claim...,Positive
1,1,"Every once in a while a movie comes, that trul...",Positive
2,2,This is a movie that only those who have felt ...,Positive
3,3,"Truly a masterpiece, The Best Hollywood film o...",Positive
4,4,Most of the time movies are anticipated like t...,Negative
...,...,...,...
95,95,"Dark, depressing and unsettling film with a ha...",Positive
96,96,I just didn't get the hype about this movie. A...,Negative
97,97,Story was just really unconvincing. Nobody rea...,Negative
98,98,JOKER is a gift to the audiences. I felt as a ...,Positive


In [ ]:
from sklearn.metrics import f1_score, accuracy_score
textblob_data = pd.read_csv("/content/joker_data1.csv")
textblob_data['polarity'] = textblob_data['review_content'].apply(lambda x: TextBlob(x).sentiment.polarity)
textblob_data['predicted sentiment'] = pd.cut(textblob_data['polarity'], bins=4, labels=[1, 2, 3, 4])
#Defining sentiment
def sentiment(x):
    if x in [1, 2]:
        return 'Negative'
    if x == 3:
        return 'Neutral'
    if x == 4:
        return 'Positive'
textblob_data['predicted sentiment'] = textblob_data['predicted sentiment'].apply(lambda x: sentiment(x))
print(textblob_data[['documentid', 'sentiment', 'predicted sentiment']].head(5))
textblob_accuracy = accuracy_score(textblob_data['sentiment'], textblob_data['predicted sentiment'])*100
textblob_f1 = f1_score(textblob_data['sentiment'], textblob_data['predicted sentiment'], average='macro')
print(' f1-score of the TextBlob:', textblob_f1)

```

   documentid sentiment predicted sentiment
0           0  positive             Neutral
1           1  positive             Neutral
2           2  positive             Neutral
3           3  positive             Neutral
4           4  positive             Neutral
 f1-score of the TextBlob: 0.0
```



In [ ]:

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()

myvader_data = pd.read_csv("/content/joker_data1.csv")
myvader_data['polarity'] = textblob_data['review_content'].apply(lambda x: vader.polarity_scores(x)['compound'])
myvader_data['predicted sentiment'] = pd.cut(myvader_data['polarity'], bins=4, labels=[1, 2, 3, 4])
myvader_data['predicted sentiment'] = myvader_data['predicted sentiment'].apply(lambda x: sentiment(x))
print(myvader_data[['documentid', 'sentiment', 'predicted sentiment']].head(5))
vader_accuracy = accuracy_score(myvader_data['sentiment'], myvader_data['predicted sentiment'])*100
vader_f1 = f1_score(myvader_data['sentiment'], myvader_data['predicted sentiment'], average='macro')
print(' f1-score of the VADER sentiment identification :', vader_f1)

```
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
   documentid sentiment predicted sentiment
0           0  positive             Neutral
1           1  positive            Positive
2           2  positive            Positive
3           3  positive            Positive
4           4  positive            Negative
 f1-score of the VADER sentiment identification : 0.0
```



In [ ]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

mysvm = pd.read_csv("/content/joker_data1.csv")
train, test = sklearn.model_selection.train_test_split(mysvm, train_size=0.6, test_size=0.1)
mypip = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=100, 
                                           learning_rate='optimal', tol=None))])

svm = mypip.fit(train['review_content'], train['sentiment'])
test['predicted sentiment'] = svm.predict(test['review_content'])
print(test[['documentid', 'sentiment', 'predicted sentiment']].head(5))

mysvm_acc = accuracy_score(test['sentiment'], test['predicted sentiment'])*100
mysvm_f1 = f1_score(test['sentiment'], test['predicted sentiment'], average='macro')

print('Accuracy of the TFIDF-based SVM:', mysvm_acc)
print('f1-score of the TFIDF-based SVM :', mysvm_f1)

```
    documentid sentiment predicted sentiment
70          70  positive            positive
96          96  positive            positive
58          58  positive            positive
15          15  positive            positive
0            0  positive            positive
Accuracy of the TFIDF-based SVM: 90.0
f1-score of the TFIDF-based SVM : 0.4736842105263158
```



Sentiment Analysis can be done by using 

1.TextBlob

2.VADER

3.TF-IDF based SVM. 

**TextBlob**:TextBlob is a popular Python library for processing textual data. It is built on top of NLTK, another popular Natural Language Processing toolbox for Python. 
TextBlob uses a sentiment lexicon (consisting of predefined words) to assign scores for each word, which are then averaged out to give an overall sentence sentiment score.

**VADER**: Valence Aware Dictionary and sEntiment Reasoner is another popular rule-based library for sentiment analysis.
Like TextBlob, it uses a sentiment lexicon that contains intensity measures for each word based on human-annotated labels. A key difference however, is that VADER was designed with a focus on social m
This means that it puts a lot of emphasis on rules that capture the essence of text typically seen on social media

**SVM**:TF-IDF is a numerical statistic that is intended to reflect how important a word is to a text document.
Support Vector Machine (SVM) is one of the most frequently used classification algorithms in text categorization.
It can deal with linear and non-linear data classification with the help of kernel function

The performance of the models can be measured using accuracy score and f1 score. 

**Analysis**:
From my analysis,TF-IDF Based SVM has given the best accuracy and f1 score, with 90% and 0.47 respectively,
which means out of the three models i have used for doing sentiment analysis.SVM is the best model for sentiment analysis. 
VADER and Textblob was not at all goodto identify.